### Hybrid Search Langchain

In [ ]:
#%pip install --upgrade --quiet  pinecone-client pinecone-text pinecone-notebooks

In [ ]:
import os
from dotenv import load_dotenv
load_dotenv()

PINECONE_API_KEY=os.getenv("PINECONE_API_KEY")

In [ ]:
from langchain_community.retrievers import PineconeHybridSearchRetriever

In [ ]:
import os
from pinecone import Pinecone,ServerlessSpec
index_name="hybrid-search-langchain-pinecone"
## initialize the Pinecone client
pc=Pinecone(api_key=PINECONE_API_KEY)

#create the index
if index_name not in pc.list_indexes().names():
    pc.create_index(
        name=index_name,
        dimension=384,  # dimensionality of dense model
        metric="dotproduct",  # sparse values supported only for dotproduct
        spec=ServerlessSpec(cloud="aws", region="us-east-1"),
    )

In [ ]:
index=pc.Index(index_name)
index

In [ ]:
## vector embedding and sparse matrix
import os
from dotenv import load_dotenv
load_dotenv()

os.environ["HF_TOKEN"]=os.getenv("HF_TOKEN")

from langchain_huggingface import HuggingFaceEmbeddings
embeddings=HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")
embeddings

BM25Encoder use TFIDF by default

In [ ]:
from pinecone_text.sparse import BM25Encoder

bm25_encoder=BM25Encoder().default()
bm25_encoder

In [ ]:
sentences=[
    "In 2023, I visited Paris",
        "In 2022, I visited New York",
        "In 2021, I visited New Orleans",

]

## tfidf values on these sentence
bm25_encoder.fit(sentences)

## store the values to a json file
bm25_encoder.dump("bm25_values.json")

# load to your BM25Encoder object
bm25_encoder = BM25Encoder().load("bm25_values.json")


#### create a retriever to have the combination of both sparse and dense vectors

In [ ]:
retriever=PineconeHybridSearchRetriever(embeddings=embeddings,sparse_encoder=bm25_encoder,index=index)

In [ ]:
retriever

In [ ]:
retriever.add_texts(
    [
    "In 2023, I visited Paris",
        "In 2022, I visited New York",
        "In 2021, I visited New Orleans",

]
)

In [ ]:
retriever.invoke("What city did i visit first")